In [1]:
import numpy as np
import xarray as xr

from google.cloud import storage
from graphcast import graphcast
from graphcast import checkpoint
from graphcast.solar_radiation import get_toa_incident_solar_radiation_for_xarray
from graphcast.data_utils import add_tisr_var, extract_input_target_times

In [2]:
gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")
dir_prefix = "graphcast/"

In [3]:
# name = list(gcs_bucket.list_blobs(prefix=dir_prefix+"dataset/"))[0].name.removeprefix(dir_prefix+"dataset/")
name = 'source-era5_date-2022-01-01_res-1.0_levels-13_steps-04.nc'
with gcs_bucket.blob(f"{dir_prefix}dataset/{name}").open("rb") as f:
    example_batch = xr.load_dataset(f).compute()
example_batch

<xarray.Dataset> Size: 132MB
Dimensions:                       (lon: 360, lat: 181, level: 13, time: 6,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 1kB 0.0 1.0 2.0 ... 358.0 359.0
  * lat                           (lat) float32 724B -90.0 -89.0 ... 89.0 90.0
  * level                         (level) int32 52B 50 100 150 ... 850 925 1000
  * time                          (time) timedelta64[ns] 48B 00:00:00 ... 1 d...
    datetime                      (batch, time) datetime64[ns] 48B 2022-01-01...
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 261kB 2.735e+04 ... -0.0...
    land_sea_mask                 (lat, lon) float32 261kB 1.0 1.0 ... 0.0 0.0
    2m_temperature                (batch, time, lat, lon) float32 2MB 250.7 ....
    mean_sea_level_pressure       (batch, time, lat, lon) float32 2MB 9.931e+...
    10m_v_component_of_wind       (batch, time, lat, lon) float32 2MB -0.4393...
    10m_u_component_of_wind       (batch, time, lat, lon) float32 2MB 1.309 ....
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 20MB ...
    geopotential                  (batch, time, level, lat, lon) float32 20MB ...
    u_component_of_wind           (batch, time, level, lat, lon) float32 20MB ...
    v_component_of_wind           (batch, time, level, lat, lon) float32 20MB ...
    vertical_velocity             (batch, time, level, lat, lon) float32 20MB ...
    specific_humidity             (batch, time, level, lat, lon) float32 20MB ...

In [4]:
with gcs_bucket.blob(f"{dir_prefix}params/GraphCast_small - ERA5 1979-2015 - resolution 1.0 - pressure levels 13 - mesh 2to5 - precipitation input and output.npz").open("rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)

In [5]:
mconfig = ckpt.model_config
tconfig = ckpt.task_config

In [6]:
era5 = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr")

In [7]:
temp = era5.rename(dict(longitude='lon', latitude='lat', time='datetime'))
temp = temp.rename_dims(dict(datetime='time'))
temp = temp.isel(time=range(90000, 90005))
temp

<xarray.Dataset> Size: 5GB
Dimensions:                                           (time: 5, lat: 721,
                                                       lon: 1440, level: 13)
Coordinates:
  * lat                                               (lat) float32 3kB 90.0 ...
  * level                                             (level) int64 104B 50 ....
  * lon                                               (lon) float32 6kB 0.0 ....
  * datetime                                          (time) datetime64[ns] 40B ...
Dimensions without coordinates: time
Data variables: (12/62)
    10m_u_component_of_wind                           (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_wind_speed                                    (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_dewpoint_temperature                           (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    above_ground                                      (time, level, lat, lon) float32 270MB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (time, lat, lon) float32 21MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    vorticity                                         (time, level, lat, lon) float32 270MB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    wind_speed                                        (time, level, lat, lon) float32 270MB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>

In [8]:
add_tisr_var(temp)

In [9]:
temp = temp[example_batch.keys()]

In [10]:
extract_input_target_times(temp, input_duration='12h', target_lead_times='6h')

TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [11]:
era5 = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2022-1h-360x181_equiangular_with_poles_conservative.zarr")
era5

<xarray.Dataset> Size: 34TB
Dimensions:                                           (time: 552264,
                                                       longitude: 360,
                                                       latitude: 181, level: 37)
Coordinates:
  * latitude                                          (latitude) float64 1kB ...
  * level                                             (level) int64 296B 1 .....
  * longitude                                         (longitude) float64 3kB ...
  * time                                              (time) datetime64[ns] 4MB ...
Data variables: (12/31)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 144GB dask.array<chunksize=(8, 360, 181), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 144GB dask.array<chunksize=(8, 360, 181), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 144GB dask.array<chunksize=(8, 360, 181), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 261kB dask.array<chunksize=(360, 181), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 261kB dask.array<chunksize=(360, 181), meta=np.ndarray>
    geopotential                                      (time, level, longitude, latitude) float32 5TB dask.array<chunksize=(8, 37, 360, 181), meta=np.ndarray>
    ...                                                ...
    total_precipitation                               (time, longitude, latitude) float32 144GB dask.array<chunksize=(8, 360, 181), meta=np.ndarray>
    type_of_high_vegetation                           (longitude, latitude) float32 261kB dask.array<chunksize=(360, 181), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 261kB dask.array<chunksize=(360, 181), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 5TB dask.array<chunksize=(8, 37, 360, 181), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 5TB dask.array<chunksize=(8, 37, 360, 181), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 5TB dask.array<chunksize=(8, 37, 360, 181), meta=np.ndarray>